# Analysis and visualization of spatial transcriptomics data

**Author:** Giovanni Palla

* This tutorial demonstrates how to work with spatial transcriptomics data within Scanpy.
* We focus on 10x Genomics [Visium](https://www.10xgenomics.com/spatial-transcriptomics/) data, and provide an example for [MERFISH](https://scanpy-tutorials.readthedocs.io/en/latest/spatial/basic-analysis.html#MERFISH-example).

In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess

In [ ]:
%%bash
pwd

In [ ]:
# sc.logging.print_header()
sc.logging.print_versions()
subprocess.run(['rm', '-f', 'None-requirements.txt'])
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3

## Reading the data

We will use a Visium spatial transcriptomics dataset of the human lymphnode, which is publicly available from the 10x genomics website: [link](https://support.10xgenomics.com/spatial-gene-expression/datasets/1.0.0/V1_Human_Lymph_Node).

The function [datasets.visium_sge()](https://scanpy.readthedocs.io/en/latest/api/scanpy.datasets.visium_sge.html) downloads the dataset from 10x Genomics and returns an `AnnData` object that contains counts, images and spatial coordinates. We will calculate standards QC metrics with [pp.calculate_qc_metrics](https://scanpy.readthedocs.io/en/latest/api/scanpy.pp.calculate_qc_metrics.html) and percentage of mitochondrial read counts per sample.

When using your own Visium data, use [sc.read_visium()](https://scanpy.readthedocs.io/en/latest/api/scanpy.read_visium.html) function to import it.

In [ ]:
adata = sc.datasets.visium_sge(sample_id="V1_Human_Lymph_Node")
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("MT-") # 'mt-' for mouse, 'MT-' for human
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

This is how the adata structure looks like for Visium data

In [ ]:
adata

## QC and preprocessing

We perform some basic filtering of spots based on total counts and expressed genes

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15, 4))
sns.distplot(adata.obs["total_counts"], kde=False, ax=axs[0])
sns.distplot(adata.obs["total_counts"][adata.obs["total_counts"] < 10000], kde=False, bins=40, ax=axs[1])
sns.distplot(adata.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[2])
sns.distplot(adata.obs["n_genes_by_counts"][adata.obs["n_genes_by_counts"] < 4000], kde=False, bins=60, ax=axs[3])

In [ ]:
sc.pp.filter_cells(adata, min_counts=5000)
sc.pp.filter_cells(adata, max_counts=35000)
adata = adata[adata.obs["pct_counts_mt"] < 20]
print(f"#cells after MT filter: {adata.n_obs}")
sc.pp.filter_genes(adata, min_cells=10)

We proceed to normalize Visium counts data with the built-in `normalize_total` method from Scanpy, and detect highly-variable genes (for later). Note that there are alternatives for normalization (see discussion in [[Luecken19](https://www.embopress.org/doi/full/10.15252/msb.20188746)], and more recent alternatives such as [SCTransform](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1874-1) or [GLM-PCA](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1861-6)).

In [ ]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)

## Manifold embedding and clustering based on transcriptional similarity

To embed and cluster the manifold encoded by transcriptional similarity, we proceed as in the standard clustering tutorial.

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, key_added="clusters")

We plot some covariates to check if there is any particular structure in the UMAP associated with total counts and detected genes.

In [ ]:
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.umap(adata, color=["total_counts", "n_genes_by_counts", "clusters"], wspace=0.4)

## Visualization in spatial coordinates

Let us now take a look at how `total_counts` and `n_genes_by_counts` behave in spatial coordinates. We will overlay the circular spots on top of the Hematoxylin and eosin stain (H&E) image provided, using the function [sc.pl.spatial](https://scanpy.readthedocs.io/en/latest/api/scanpy.pl.spatial.html).

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(adata, img_key="hires", color=["total_counts", "n_genes_by_counts"])

The function [sc.pl.spatial](https://scanpy.readthedocs.io/en/latest/api/scanpy.pl.spatial.html) accepts 4 additional parameters:

* `img_key`: key where the img is stored in the `adata.uns` element
* `crop_coord`: coordinates to use for cropping (left, right, top, bottom)
* `alpha_img`: alpha value for the transcparency of the image
* `bw`: flag to convert the image into gray scale  

Furthermore, in [sc.pl.spatial](https://scanpy.readthedocs.io/en/latest/api/scanpy.pl.spatial.html), the `size` parameter changes its behaviour: it becomes a scaling factor for the spot sizes.

Before, we performed clustering in gene expression space, and visualized the results with UMAP. By visualizing clustered samples in spatial dimensions, we can gain insights into tissue organization and, potentially, into inter-cellular communication.

In [ ]:
sc.pl.spatial(adata, img_key="hires", color="clusters", size=1.5)

Spots belonging to the same cluster in gene expression space often co-occur in spatial dimensions. For instance, spots belonging to cluster 5 are often surrounded by spots belonging to cluster 0.

We can zoom in specific regions of interests to gain qualitative insights. Furthermore, by changing the alpha values of the spots, we can visualize better the underlying tissue morphology from the H&E image.

In [ ]:
sc.pl.spatial(adata, img_key="hires", color="clusters", groups=["0", "5"], crop_coord=[1200, 1700, 1900, 1000], alpha=0.5, size=1.3)

## Cluster marker genes

Let us further inspect cluster 5, which occurs in small groups of spots across the image.

Compute marker genes and plot a heatmap with expression levels of its top 10 marker genes across clusters.

In [ ]:
sc.tl.rank_genes_groups(adata, "clusters", method="t-test")
sc.pl.rank_genes_groups_heatmap(adata, groups="5", n_genes=10, groupby="clusters")

We see that *CR2* recapitulates the spatial structure.

In [ ]:
sc.pl.spatial(adata, img_key="hires", color=["clusters", "CR2"])

## Spatially variable genes

Spatial transcriptomics allows researchers to investigate how gene expression trends varies in space, thus identifying spatial patterns of gene expression. For this purpose, we use SpatialDE [Svensson18](https://www.nature.com/articles/nmeth.4636) ([code](https://github.com/Teichlab/SpatialDE)), a Gaussian process-based statistical framework that aims to identify spatially variable genes:


Recently, other tools have been proposed for the identification of spatially variable genes, such as:

* SPARK [paper](https://www.nature.com/articles/s41592-019-0701-7#Abs1) - [code](https://github.com/xzhoulab/SPARK)
* trendsceek [paper](https://www.nature.com/articles/nmeth.4634) - [code](https://github.com/edsgard/trendsceek)
* HMRF [paper](https://www.nature.com/articles/nbt.4260) - [code](https://bitbucket.org/qzhudfci/smfishhmrf-py/src/default/)

First, we convert normalized counts and coordinates to pandas dataframe, needed for inputs to spatialDE.

Running SpatialDE takes considerable time.

In [ ]:
import SpatialDE

In [ ]:
%%time
counts = pd.DataFrame(adata.X.todense(), columns=adata.var_names, index=adata.obs_names)
coord = pd.DataFrame(adata.obsm['spatial'], columns=['x_coord', 'y_coord'], index=adata.obs_names)
results = SpatialDE.run(coord, counts)

We concatenate the results with the DataFrame of annotations of variables: `adata.var`.

In [ ]:
results.index = results["g"]
adata.var = pd.concat([adata.var, results.loc[adata.var.index.values, :]], axis=1)

Then we can inspect significant genes that varies in space and visualize them with `sc.pl.spatial` function.

In [ ]:
results.sort_values("qval").head(10)

In [ ]:
sc.pl.spatial(adata, img_key="hires", color=["COL1A2", "SYPL1"], alpha=0.7)

## MERFISH example

In case you have spatial data generated with FISH-based techniques, just read the cordinate table and assign it to the `adata.obsm` element. 

Let's take a look at the example from [Xia et al. 2019](https://www.pnas.org/content/116/39/19490.abstract).

First, we need to download the coordinate and counts data from the [original publication](https://www.pnas.org/content/116/39/19490/tab-figures-data).

In [ ]:
subprocess.run(['mkdir', '-p', './downloaded_data'])
subprocess.run(['cp', '-n', '../data/pnas.1912459116.sd15.xlsx', 'downloaded_data'])
subprocess.run(['cp', '-n', '../data/pnas.1912459116.sd12.csv', 'downloaded_data'])

And read the data in a AnnData object.

In [ ]:
coordinates = pd.read_excel("./downloaded_data/pnas.1912459116.sd15.xlsx", index_col=0)
counts = sc.read_csv("./downloaded_data/pnas.1912459116.sd12.csv").transpose()

In [ ]:
adata_merfish = counts[coordinates.index, :]
adata_merfish.obsm["spatial"] = coordinates.to_numpy()

We will perform standard preprocessing and dimensionality reduction.

In [ ]:
sc.pp.normalize_per_cell(adata_merfish, counts_per_cell_after=1e6)
sc.pp.log1p(adata_merfish)
sc.pp.pca(adata_merfish, n_comps=15)
sc.pp.neighbors(adata_merfish)
sc.tl.umap(adata_merfish)
sc.tl.leiden(adata_merfish, key_added="clusters", resolution=0.5)

The experiment consisted in measuring gene expression counts from a single cell type (cultured U2-OS cells). Clusters consist of cell states at different stages of the cell cycle. We don't expect to see specific structure in spatial dimensions given the experimental setup.

We can visualize the clusters obtained from running Leiden in UMAP space and spatial coordinates like this.

In [ ]:
adata_merfish

In [ ]:
sc.pl.umap(adata_merfish, color="clusters")
sc.pl.embedding(adata_merfish, basis="spatial", color="clusters")

#### We hope you found the tutorial useful!

[Report back to us](https://github.com/theislab/scanpy/issues/new?labels=enhancement&template=enhancement-request.md) which features/external tools you would like to see in Scanpy. 

We are extending Scanpy and AnnData to support other spatial data types, such as **Imaging Mass Cytometry** and extend data structure to support spatial graphs and additional features. Stay tuned!